In [ ]:
!pip install sentencepiece
!pip install transformers

     |████████████████████████████████| 1.2MB 10.9MB/s 
     |████████████████████████████████| 2.3MB 9.9MB/s 
     |████████████████████████████████| 3.3MB 34.4MB/s 
     |████████████████████████████████| 901kB 36.0MB/s 


In [ ]:
# !pip install sentencepiece
# !pip install transformers

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from tqdm.notebook import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
import pandas as pd
import nltk
import datetime
import numpy as np
# from transformers import 
from transformers import M2M100Config, M2M100ForConditionalGeneration, M2M100Tokenizer, AdamW


cuda


In [ ]:
# MOUNT DRIVE TO SAVE OUTPUTS

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# https://colab.research.google.com/drive/1-JIJlao4dI-Ilww_NnTc0rxtp-ymgDgM?usp=sharing
# https://huggingface.co/transformers/custom_datasets.html#additional-resources


In [ ]:
!nvidia-smi
# https://huggingface.co/transformers/model_doc/m2m_100.html

Wed May 12 18:20:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# get data
! git clone https://github.com/zachary-m/gc.git
df = pd.read_pickle("/content/gc/train_test_val.pkl")
(df.src + "\t" + df.target).to_csv("bibheb-modernheb.txt", index=False, header=None)

Cloning into 'gc'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [ ]:
model = M2M100ForConditionalGeneration.from_pretrained('facebook/m2m100_418M')
tokenizer = M2M100Tokenizer.from_pretrained('facebook/m2m100_418M', src_lang="he", tgt_lang="he")

In [ ]:
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
model = model.train()

In [ ]:
train = df.loc[df["data_type"] == "train", ["src", "target"]]
val = df.loc[df["data_type"] == "val", ["src", "target"]]
test = df.loc[df["data_type"] == "test", ["src", "target"]]

In [ ]:
# HOW DO I GET THIS TOWORK???
# from datasets import load_metric
# metric = load_metric("bleu")

In [ ]:
RUN_NAME = "training"
LOG_FILE_NAME = 'log_hugface_model.csv'
DRIVE_PATH = "/content/gdrive/MyDrive/"
LOG_FILE_FULL = DRIVE_PATH + LOG_FILE_NAME

tokenizer.src_lang = "he"
losses = []
bleus = []
best_bleu = -1
best_parameters = None
for epoch in list(range(10)):
  model.train() # am I supposed to do this each time?
  epoch_loss = []
  bleus_epoch = []
  # TRAIN
  print("-"*8 + " Epoch {} ".format(epoch) + "-"*8)
  for i, row in tqdm(list(train.sample(frac=1).iterrows())):  # maybe increase the batch size?
    src, target = row["src"], row["target"]
    model_inputs = tokenizer(src, return_tensors="pt").to(device)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target, return_tensors="pt").input_ids
    labels = labels.to(device)
    # print(labels)
    optimizer.zero_grad()
    loss = model(**model_inputs, labels=labels)# forward pass
    loss.loss.backward()
    optimizer.step()
    epoch_loss.append(loss.loss.item()) # add to loss records
    if i % 1000 == 0:
      print("Epoch: {}  Loss: {}".format(epoch, loss.loss))

  # RECORD INFO!
  dt = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
  df_vls = [epoch, "train", "loss", dt, RUN_NAME] + epoch_loss
  pd.DataFrame(df_vls).T.to_csv(LOG_FILE_FULL , mode='a', index=False, header=False)

  print("Epoch: {}  Mean Training Loss: {}".format(epoch, np.mean(epoch_loss)))
  losses.append(epoch_loss) # keep track of all losses
  model.eval()
  
  # VAL BLEU!
  # THIS SHOULD BE MUCH FASTER, HOW TONOT USE NLTK??
  for i, row in tqdm(list(val.iterrows())):
    optimizer.zero_grad(False)
    src, target = row["src"], row["target"]
    encoded_hi = tokenizer(src, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("he"))
    encode_target = tokenizer(target, return_tensors="pt").to(device)
    encode_target = encode_target['input_ids'].squeeze().cpu().detach().numpy()
    predicted = generated_tokens.squeeze().cpu().detach().numpy()
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([encode_target], predicted, weights = [1])
    bleus_epoch.append(BLEUscore) # record in the epoch



  # RECORD INFO!
  dt = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
  df_vls = [epoch, "val", "BLUE", dt, RUN_NAME] + bleus_epoch
  pd.DataFrame(df_vls).T.to_csv(LOG_FILE_FULL, mode='a', index=False, header=False)


  bleus.append(bleus_epoch) # record all
  mean_bleu = np.mean(bleus_epoch)

  # record model parameters of best model
  if  mean_bleu > best_bleu:
    best_bleu = mean_bleu
    best_parameters = model.parameters()
    # model.save
  
  print("Epoch: {}  Mean Val BLEU {}".format(epoch, mean_bleu))



In [ ]:
  df_vls = [epoch, "train", "loss", dt, RUN_NAME] + epoch_loss
  pd.DataFrame(df_vls).T.to_csv('/content/gdrive/MyDrive/log_hugface_model.csv', mode='a', index=False, header=False)

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
pd.DataFrame(df_vls)

,0
0,epoch 0
1,train
2,loss
3,"05/12/2021, 18:25:12"
4,TEST
5,1.53829
6,1.38626
7,0.0727819
8,0.180002
9,1.11663


# get model outputs

In [ ]:
model.eval()
encoded_hi = tokenizer(src, return_tensors="pt").to(device)
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("he"))
print("orignal")
print(src)
print("TARGET")
print(target)
print("RESULT")
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

orignal
ובהר בבקעה יצאים ותהמת עינת מים נחלי ארץ טובה אל ארץ מביאך אלהיך יהוה כי
TARGET
ובהר בבקעה שנובעים ותהומות מעינות מים נחלי ארץ טובה לארץ אותך מביא אלהיך יהוה כי
RESULT


['ובהר בבקעה יוצאים ותהומות מעינות מים נחלי ארץ טובה לארץ אותך מביא אלהיך יהוה כי']

# Trying to get it to train in batches

In [ ]:
tokenizer.src_lang = "he"
losses = []
bleus = []

best_bleu = -1
best_parameters = None
for epoch in list(range(10)):
  model.train() # am I supposed to do this each time?
  epoch_loss = []
  bleus_epoch = []
  # TRAIN
  train_ = train.sample(frac=1)
  index_batches = np.array_split(train.index, 230) # batch size of 32
  for indexes  in tqdm(index_batches):  # maybe increase the batch size?
    train_batch = train_.iloc[indexes]
    src, target = train_batch[["src"]].values, train_batch[["target"]].values  # get these columns and the numpy arrays for them
    model_inputs = tokenizer(src, return_tensors="pt").to(device)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target, return_tensors="pt").input_ids
    labels = labels.to(device)
    # print(labels)
    optimizer.zero_grad()
    loss = model(**model_inputs, labels=labels)# forward pass
    loss.loss.backward()
    optimizer.step()
    # break
    epoch_loss.append(loss.loss.item()) # add to loss records
    if i % 1000 == 0:
      print("Epoch: {}  Loss: {}".format(epoch, loss.loss))
    break

  print("Epoch: {}  Mean Training Loss: {}".format(epoch, np.mean(epoch_loss)))
  losses.append(epoch_loss) # keep track of all losses
  model.eval()
  
  # VAL BLEU!
  
  for i, row in tqdm(list(val.iterrows())):
    optimizer.zero_grad(False)
    src, target = row["src"], row["target"]
    encoded_hi = tokenizer(src, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("he"))
    encode_target = tokenizer(target, return_tensors="pt").to(device)
    encode_target = encode_target['input_ids'].squeeze().cpu().detach().numpy()
    predicted = generated_tokens.squeeze().cpu().detach().numpy()
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([encode_target], predicted, weights = [1])
    bleus_epoch.append(BLEUscore) # record in the epoch
    break

  bleus.append(bleus_epoch) # record all
  mean_bleu = np.mean(bleus_epoch)

  # record model parameters of best model
  if  mean_bleu > best_bleu:
    best_bleu = mean_bleu
    best_parameters = model.parameters()
  
  print("Epoch: {}  Mean Val BLEU {}".format(epoch, mean_bleu))

